# MÃ HÓA BIẾN SỐ ĐỊNH LƯỢNG VỚI SCIKIT-LEARN

Trong biên tập, xử lý và phân tích dữ liệu, một số trường hợp đòi hỏi nhà phân tích phải biến đổi một biến số liên tục sang biến số phân loại có từ hai hoặc nhiều nhóm.  
Ví dụ: biến số BMI có thể biến đổi thành biến số phân loại tình trạng dinh dưỡng gồm các nhóm: thiếu cân (< 18,5), bình thường (18,50 - 22,99), thừa cân (23,00 - 24,99), béo phì độ I (25,00 - 29,99), béo phì độ II	(30,00 - 39,99), béo phì độ III	(≥ 40). Hoặc có thể biến đổi thành biến số Béo phì: Có (>= 25) và Không (< 25).

In [1]:
# Cho bộ dữ liệu mô phỏng gồm 3 biến số: gioi_tinh, dan_toc, bmi
import pandas as pd
import numpy as np

data= pd.read_csv('data/bmi-simulation.csv', index_col= 0)
data.head()

,gioi_tinh,ton_giao,bmi
0,Nam,Khong,21.57
1,Nam,Cong_Giao,20.95
2,Nu,Khong,25.33
3,Nu,Khong,21.04
4,Nam,Khong,21.44


## 1. Chuyển đổi biến liên tục thành biến nhị phân

In [2]:
from sklearn.preprocessing import Binarizer
from sklearn.compose import ColumnTransformer
binarizer= Binarizer(threshold= 24.9) # <= threshold = 0, > threshold = 1
binarize_transformer= ColumnTransformer(transformers=[('Binarizer', binarizer, ['bmi'])])
data['beo_phi']= binarize_transformer.fit_transform(data)
data.head()

,gioi_tinh,ton_giao,bmi,beo_phi
0,Nam,Khong,21.57,0.0
1,Nam,Cong_Giao,20.95,0.0
2,Nu,Khong,25.33,1.0
3,Nu,Khong,21.04,0.0
4,Nam,Khong,21.44,0.0


Có thể map với 1 functin thông qua FunctionTransformer như ở mục 2

## 2. Chuyển đổi biến liên tục thành biến phân loại (có thứ tự)

### 2.1. Chuyển đổi theo các khoảng cho trước
Chuyển đổi biến số bmi thành biến số tình trạng dinh dưỡng:
- Thiếu cân (< 18,5)
- Bình thường (18,50 - 22,99)
- Thừa cân (23,00 - 24,99)
- Béo phì độ I (25,00 - 29,99)
- Béo phì độ II (30,00 - 39,99)
- Béo phì độ III (≥ 40)

In [3]:
# Function xác định tình trạng dinh dưỡng của mỗi người
def tinh_trang_dinh_duong(bmi):
    if bmi < 18.5:
        ttdd= 'Thieu_can'
    elif bmi >= 18.5 and bmi < 23:
        ttdd= 'Binh_thuong'
    elif bmi >= 23 and bmi < 25:
        ttdd= 'Thua_can'
    elif bmi >= 25 and bmi < 30:
        ttdd= 'Beo_phi_do_1'
    elif bmi >= 30 and bmi < 40:
        ttdd= 'Beo_phi_do_2'
    else:
        ttdd= 'Beo_phi_do_3'
    return ttdd


In [4]:
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer

tt_dinh_duong_function_transformer= FunctionTransformer(func= lambda x: x.applymap(tinh_trang_dinh_duong))
tt_dinh_duong_transformer= ColumnTransformer([('Tinh trang dinh duong',
                                               tt_dinh_duong_function_transformer,
                                               ['bmi'])])
data['tt_dinh_duong']= tt_dinh_duong_transformer.fit_transform(data)
data.head()

,gioi_tinh,ton_giao,bmi,beo_phi,tt_dinh_duong
0,Nam,Khong,21.57,0.0,Binh_thuong
1,Nam,Cong_Giao,20.95,0.0,Binh_thuong
2,Nu,Khong,25.33,1.0,Beo_phi_do_1
3,Nu,Khong,21.04,0.0,Binh_thuong
4,Nam,Khong,21.44,0.0,Binh_thuong


### 2.2. Chuyển đổi dựa vào số lượng khoảng giá trị
Chuyển đổi thành một số lượng các nhóm mà mỗi nhóm có chiều rộng bằng nhau trong phạm vi của biến số.

In [5]:
def assign_labels_function(x): #x: arr
    label= {0: 'Thap', 1: 'Trung_binh',2: 'Cao',3: 'Rat_cao'}
    ls_labels= []
    for i in x:
        ls_labels.append(label[i[0]])
    return np.array(ls_labels).reshape(x.shape)

In [6]:
from sklearn.preprocessing import KBinsDiscretizer, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

uniform_k_bins_discretizer= KBinsDiscretizer(n_bins= 4, strategy= 'uniform', encode= 'ordinal')
assign_labels= FunctionTransformer(func= assign_labels_function)

pipe= Pipeline(steps= [('uniform_k_bins', uniform_k_bins_discretizer),
                       ('assign_labels', assign_labels)])

uniform_k_bins_discretizer_transformer= ColumnTransformer([('uniform_k_bins_discretizer',
                                                            pipe,
                                                            ['bmi'])])
data['ordinal_var_1']= uniform_k_bins_discretizer_transformer.fit_transform(data)

In [7]:
data.head()

,gioi_tinh,ton_giao,bmi,beo_phi,tt_dinh_duong,ordinal_var_1
0,Nam,Khong,21.57,0.0,Binh_thuong,Trung_binh
1,Nam,Cong_Giao,20.95,0.0,Binh_thuong,Trung_binh
2,Nu,Khong,25.33,1.0,Beo_phi_do_1,Cao
3,Nu,Khong,21.04,0.0,Binh_thuong,Trung_binh
4,Nam,Khong,21.44,0.0,Binh_thuong,Trung_binh


Chuyển đổi dựa vào khoảng phân vị thì sử dụng strategy = 'quantile'  
Lớp này còn hỗ trợ phân cụm bằng thuật toán Kmean khi truyền đối số 'kmeans' vào cho tham số strategy.